***Import necessary libraries***

In [1]:
import ast
import re
from tqdm import tqdm
import json
import pandas as pd
import numpy as np

c:\Users\antoi\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\antoi\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
c:\Users\antoi\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


## **Files Name**

In [2]:

# Specify the path to your file
input_filename = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/segment_entier/contained_all_segments.txt"
output_filename = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/segment_entier/filtered_segments.txt"
output_filename_20 = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/segment_entier/filtered_segments_20.txt"

output_file = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/segment_entier/contained_all_segments_ordered.txt"

test_input = "data/segment_test_filter.txt"
test_output = "data/segment_test_filter_output.txt"

## Extract upper left corner of the image

In [ ]:
# Open the input file for reading
with open(input_filename, 'r') as input_file:
    # Open the output file for writing
    with open(output_filename, 'w') as output_file:
        for line_num, line in enumerate(input_file, 1):
            # Extract relevant information from the line
            segment_info = line.strip().split(', ')
            print(segment_info)
            segment_id = int(segment_info[0].split(': ')[1])
            print(segment_id)

            # Extract the pixel coordinates using regular expression
            pixels_str = re.search(r'Pixels: \[\[(.*?)\]\]', line)
            print(pixels_str)
            if pixels_str:
                pixels_str = pixels_str.group(1)
                pixels = [list(map(int, coord.split(', '))) for coord in pixels_str.split('], [')]

                # Check if the segment is in the upper-left corner
                if all(0 <= x <= 2256 and 0 <= y <= 2591 for x, y in pixels):
                    # Write the line to the new file
                    output_file.write(line)

            # Update Progress Bar
print(f"\nFiltered segments saved to {output_filename}")

## Verifications

Check if the max and min values correspond to the x and y boundaries

In [ ]:
import pandas as pd

# Load the filtered segments into a DataFrame
filtered_df = pd.read_csv(output_filename_20, sep=',', header=None, low_memory=False, on_bad_lines='skip')
# Drop columns 0, 1, and 2
filtered_df = filtered_df.drop(columns=[0, 1, 2])
# Concatenate the remaining columns into one
filtered_df['Concatenated Pixels'] = filtered_df.apply(lambda row: ''.join(map(str, row)), axis=1)
filtered_df = filtered_df[['Concatenated Pixels']]

In [8]:
# Remove the 'Pixels: ' prefix and convert the remaining string to a list
filtered_df['Pixels'] = filtered_df['Concatenated Pixels'].str.replace('Pixels: ', '')
filtered_df = filtered_df[['Pixels']]

In [9]:
def convert_to_tuple_list(pixel_str):
    if isinstance(pixel_str, float) and np.isnan(pixel_str):
        return []
    pixel_str = pixel_str.replace('[', '').replace(']', '').replace('nan', '')
    pixel_list = pixel_str.split()
    return [(int(pixel_list[i]), int(pixel_list[i + 1])) for i in range(0, len(pixel_list), 2) if 'nan' not in pixel_list[i]]

filtered_df['Pixels'] = filtered_df['Pixels'].apply(convert_to_tuple_list)

In [10]:
max_x = max_y = 0

for pixel_list in filtered_df['Pixels']:
    for pixel in pixel_list:
        x, y = pixel
        max_x = max(max_x, x)
        max_y = max(max_y, y)

print(f"Max X: {max_x}, Max Y: {max_y}")

Max X: 225, Max Y: 259


## Create the perimeter

We want to create a new column giving the pixels of the perimeter of each segment, ordered clockwise.

In [14]:
# Read the file and split lines
with open(test_input, 'r') as file:
    lines = file.read().splitlines()

Create the dataframe from the file

In [15]:
# Initialize empty lists to store data
segment_ids, polygon_ids, class_ids, pixels_list = [], [], [], []

# Iterate through lines and extract information
for line in lines:
    if line.strip():  # Check if the line is not empty
        segments = line.split(', ')
        # Extract values using string manipulation
        segment_id = int(segments[0].split(': ')[1])
        polygon_id = 0
        class_id = 0
        # pixels are the concatenation of all other elements after the first 3
        # remove the 'Pixels: '
        pixels_str = ', '.join(segments[3:])
        pixels_str = pixels_str.replace('Pixels: ', '').replace('[', '').replace(']', '').replace(',', '').split()
        pixels_str = [(int(pixels_str[i]), int(pixels_str[i + 1])) for i in range(0, len(pixels_str), 2)]
        
        # Append values to lists
        segment_ids.append(segment_id)
        polygon_ids.append(polygon_id)
        class_ids.append(class_id)
        pixels_list.append(pixels_str)

# Create DataFrame
df = pd.DataFrame({
    'Segment ID': segment_ids,
    'Polygon ID': polygon_ids,
    'Class ID': class_ids,
    'Pixels': pixels_list
})

Create the new column

First get the perimeter

In [16]:
def get_perimeter_pixels(pixels):
    rows = {}
    for pixel in pixels:
        row, col = pixel
        if row not in rows:
            rows[row] = []
        rows[row].append(col)
    perimeter_pixels = []
    for row, cols in rows.items():
        perimeter_pixels.append((row, min(cols)))
        perimeter_pixels.append((row, max(cols)))
    return perimeter_pixels

In [17]:
df['Perimeter Pixels'] = df['Pixels'].apply(get_perimeter_pixels)


Then order the perimeter clockwise

In [18]:
def order_pixels_clockwise(pixels):
    # Calculate the centroid of the pixels
    centroid = [sum(pixel[0] for pixel in pixels)/len(pixels), sum(pixel[1] for pixel in pixels)/len(pixels)]
    
    # Sort the pixels by the angle each pixel makes with the centroid
    sorted_pixels = sorted(pixels, key=lambda pixel: np.arctan2(pixel[1] - centroid[1], pixel[0] - centroid[0]))
    
    # Convert the result back to a list of tuples
    return [tuple(pixel) for pixel in sorted_pixels]

In [19]:
df['Perimeter Pixels'] = df['Perimeter Pixels'].apply(lambda x: order_pixels_clockwise(np.array(x)))

In [21]:
df

,Segment ID,Polygon ID,Class ID,Pixels,Perimeter Pixels
0,0,0,0,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 4), (4, 4..."
1,1,0,0,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (4, 2), (4, 2), (2, 4..."
2,2,0,0,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 4..."
3,3,0,0,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 4..."
4,4,0,0,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 4..."
5,5,0,0,"[(0, 2), (2, 0), (2, 4), (4, 2)]","[(2, 0), (4, 2), (4, 2), (2, 4), (0, 2), (0, 2)]"


Now we can save the dataframe

In [12]:
# save the dataframe in a csv file
df.to_csv(test_output, index=False)